In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import * #chi2_contingency로 카이제곱통계량계산
from scipy.stats import norm

import keyword
import random

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math



# Lecture 11 회귀분석(2)
## 3 행렬을 이용한 계수 추정

- 필참!!

In [2]:
df=pd.read_csv('c:/temp/education2020.csv')
df1=df[['income','age','education_year']]
edu_model=ols('income~age+education_year',data=df1).fit()
print(edu_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1773.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):               0.00
Time:                        19:52:53   Log-Likelihood:            -1.7980e+05
No. Observations:               18064   AIC:                         3.596e+05
Df Residuals:                   18061   BIC:                         3.596e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        322.1142    277.944      1.

- 표에
- coef, std err, t, p 가 무슨 뜻인지 알아야 함
- 그중에 coef, std err은 행렬로도 직접 구할 수 있음!!

In [3]:
# 행렬을 이용해서 같은 결과를 도출해보자
# 오잉?? 강의가 똑같은데?
# 0522-2 강의 시작

## 4 제곱항과 더미변수 추가

In [4]:
# age^2을 회귀모형에 포함하자
df1['age2']=df1['age']**2
df1.head()

,income,age,education_year,age2
0,6593,34,16,1156
1,17720,45,21,2025
2,908,73,6,5329
3,2748,58,12,3364
4,1015,27,12,729


In [5]:
edu_model1=ols('income~education_year+age+age2',data=df1).fit()
print(edu_model1.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.187
Model:                            OLS   Adj. R-squared:                  0.187
Method:                 Least Squares   F-statistic:                     1387.
Date:                Wed, 12 Jun 2024   Prob (F-statistic):               0.00
Time:                        20:06:16   Log-Likelihood:            -1.7954e+05
No. Observations:               18064   AIC:                         3.591e+05
Df Residuals:                   18060   BIC:                         3.591e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -9472.0780    511.480    -18.

In [6]:
# age**2의 coef는 마이너스, age의 coef는 플러스로 나타남
# 해석: 이차함수 극값의 엑스좌표가 1사분면에 존재한다.
# ㅇㅎㅇㅎ

In [8]:
# Q7: 소득이 최대가 되는 나이 = -2a/b
-380/(2*-3.439)

55.248618784530386

In [7]:
# 행렬로도 이 모형 coef 구할 수 있음
# 해보기!!

In [9]:
df

,Unnamed: 0,year,metro,id,sex,age,number,education,marriage,asset,debt,income,income_d,industry,job,house,education_year
0,0,2020,G1,10000112,1,34,3,6,2,112000,54500,6593,4599,F,3,2,16
1,1,2020,G1,10000132,2,45,2,8,2,42500,17500,17720,15257,J,2,3,21
2,2,2020,G1,10000162,2,73,1,2,3,5712,0,908,725,T,4,2,6
3,3,2020,G1,10000182,1,58,2,4,2,14870,0,2748,2431,C,5,2,12
4,4,2020,G1,10000192,2,27,1,4,1,814,0,1015,893,R,2,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18059,18059,2020,G2,993800291,1,42,3,6,2,6705,2200,179,-390,NaN,NaN,3,16
18060,18060,2020,G2,994800251,1,49,5,6,2,130180,22000,15454,11784,O,3,1,16
18061,18061,2020,G2,994800261,1,57,5,6,2,265226,29000,15098,11560,NaN,NaN,1,16
18062,18062,2020,G2,994800291,2,53,2,7,4,109695,25000,9114,7074,O,2,2,18


In [10]:
df.value_counts()

Unnamed: 0  year  metro  id         sex  age  number  education  marriage  asset   debt   income  income_d  industry  job  house  education_year
0           2020  G1     10000112   1    34   3       6          2         112000  54500  6593    4599      F         3    2      16                1
12062       2020  G2     650600331  1    64   2       4          2         24510   200    6267    5694      F         3    1      12                1
12047       2020  G2     650000282  1    66   3       2          2         19200   36     7147    6458      N         9    1      6                 1
12050       2020  G2     650000322  1    49   4       4          2         32617   8350   3034    2364      F         7    1      12                1
12051       2020  G2     650000332  1    32   4       5          2         9960    3700   3632    3542      G         5    3      14                1
                                                                                                         

In [11]:
# job 카테고리에 1, 2, 3, 4, 5, 6, 7, 8, 9, A 라는 카테고리까지 있었음
# 'A'라는 job을 삭제하는 코드가 교수님 lecture note에 있음

In [12]:
df1=df.loc[df['job']!='A']
df1.head()

,Unnamed: 0,year,metro,id,sex,age,number,education,marriage,asset,debt,income,income_d,industry,job,house,education_year
0,0,2020,G1,10000112,1,34,3,6,2,112000,54500,6593,4599,F,3,2,16
1,1,2020,G1,10000132,2,45,2,8,2,42500,17500,17720,15257,J,2,3,21
2,2,2020,G1,10000162,2,73,1,2,3,5712,0,908,725,T,4,2,6
3,3,2020,G1,10000182,1,58,2,4,2,14870,0,2748,2431,C,5,2,12
4,4,2020,G1,10000192,2,27,1,4,1,814,0,1015,893,R,2,3,12


#### X에 더미변수를 추가
- 지금까지 age, education은 연속형변수였다

In [13]:
df2=df1[['income','age','education_year']]
df2.head()

,income,age,education_year
0,6593,34,16
1,17720,45,21
2,908,73,6
3,2748,58,12
4,1015,27,12


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18039 entries, 0 to 18063
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      18039 non-null  int64 
 1   year            18039 non-null  int64 
 2   metro           18039 non-null  object
 3   id              18039 non-null  int64 
 4   sex             18039 non-null  int64 
 5   age             18039 non-null  int64 
 6   number          18039 non-null  int64 
 7   education       18039 non-null  int64 
 8   marriage        18039 non-null  int64 
 9   asset           18039 non-null  int64 
 10  debt            18039 non-null  int64 
 11  income          18039 non-null  int64 
 12  income_d        18039 non-null  int64 
 13  industry        13189 non-null  object
 14  job             13189 non-null  object
 15  house           18039 non-null  int64 
 16  education_year  18039 non-null  int64 
dtypes: int64(14), object(3)
memory usage: 2.5+ MB


In [15]:
# 지금 df1에 성별 sex 변수있음
# 이걸 regression에 사용하기 위해 '더미변수'로 만들자
# sex_dum1, sex_dum2

In [19]:
sex_dummies=pd.get_dummies(df1['sex']).rename(columns=lambda x: 'sex'+str(x))
# 그럼 알아서 더미 두개로 만들어진다
# .rename ~~ : 칼럼이름 지정
# lambda x: 람다 함수를 사용한다... 뭔소리야
sex_dummies
# 1: 남자, 0: 여자

,sex1,sex2
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
18059,1,0
18060,1,0
18061,1,0
18062,0,1


In [23]:
df3=pd.concat([df2,sex_dummies], axis=1)
# df2와 더미변수를 합친 데이터프레임을 만든다
# axis=1 : 칼럼 방향, 옆으로 붙이자
df3

,income,age,education_year,sex1,sex2
0,6593,34,16,1,0
1,17720,45,21,0,1
2,908,73,6,0,1
3,2748,58,12,1,0
4,1015,27,12,0,1
...,...,...,...,...,...
18059,179,42,16,1,0
18060,15454,49,16,1,0
18061,15098,57,16,1,0
18062,9114,53,18,0,1


In [25]:
# df3를 이용해서 sex 변수가 포함된 다중선형회귀분석을 실행한다

In [22]:
# 여기까지